In [ ]:
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Importing Necessary Libraries**

In [ ]:
import pandas as pd
import re
import nlpaug.augmenter.word as naw
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report

**Loading the Dataset**

In [ ]:
#reading the dataset in csv format

data = pd.read_csv('/content/drive/MyDrive/Datasets/gittercom_annotated_data.csv')

**Text Preprocessing**


In [ ]:
def text_preprocessing(text):
    #removing punctuation from text data
    text = re.sub(r'[^\w\s]', '', text)

    #removing special symbols from text data
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)

    #replacing usernames in text data with '<username>'
    text = re.sub(r'@[^\s]+', '<username>', text)

    #replacing url in text data with '<url>' 
    text = re.sub(r'http\S+|www\S+|\S+\.com\S+', '<url>', text)
    
    #replacing code snippets in text data with '<code>' 
    text = re.sub(r'`[^`]+`', '<code>', text)
    
    return text

**Applying Text Preprocessing to the Text Data**

In [ ]:
#applying preprocessing techniques to the text data ('message' column holds all textual communications,i.e., text data)

data['message'] = data['message'].apply(text_preprocessing)

**Counting the Total Number of Instances of Our Dataset**

In [ ]:
data.shape[0]

1000

In [ ]:
#exploring first five rows of our dataset

data.head()

,Channel,message,category
0,Cucumber,Hi Team I just recently upgraded our cucumberj...,fear
1,Cucumber,So github is trying to replace irc P,fear
2,Cucumber,aslakhellesoy Thanks seems like I was using o...,joy
3,Cucumber,Sidkiyassine just call the methods directly,surprise
4,Cucumber,Hello guys need a helpI want to call the run m...,fear


In [ ]:
#exploring the instances of message column that holds text data

data['message']

0      Hi Team I just recently upgraded our cucumberj...
1                   So github is trying to replace irc P
2      aslakhellesoy Thanks  seems like I was using o...
3            Sidkiyassine just call the methods directly
4      Hello guys need a helpI want to call the run m...
                             ...                        
995    Besides the unzip issue which I saw you solved...
996    Hopefully now that the unzip issue is fixed we...
997     I can see that being a problem with my community
998                       which would slow down installs
999         Thanks yet again Ill try your suggestion now
Name: message, Length: 1000, dtype: object

**Splitting the Dataset for Classification**

In [ ]:
X = data['message']
y = data['category']

#splitting the data into training and testing sets (70% for training and 30% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


**Data Augmentation of training data using Synonym**

In [ ]:
augmented_data = []
augmented_category = []
no_of_targeted_augmentation = 30000

#initializing SynonymAug class of nlp aug library that leverage semantic meaning to substitute word using synonym
data_augmentation = naw.SynonymAug(aug_src='wordnet')

while len(augmented_data) < no_of_targeted_augmentation:
    for message, category in zip(X_train, y_train):
        augmented_text = data_augmentation.augment(message)
        augmented_data.append(augmented_text)
        augmented_category.append(category)
        if len(augmented_data) == no_of_targeted_augmentation:
            break

In [ ]:
X_train = X_train.tolist() + augmented_data[:no_of_targeted_augmentation]
y_train = y_train.tolist() + augmented_category[:no_of_targeted_augmentation]

**Counting the Total Number of Instances of training data After Data Augmentation**

In [ ]:
len(X_train)

30700

**Dropping Any Possible Duplicate Instance(s) of Text Data Due to Performing Data Augmentation**

In [ ]:
data.drop_duplicates(subset='message', inplace=True)

In [ ]:
len(X_train)

30700

**Exploring Augmented Text Data**

In [ ]:
for message, category in zip(X_train, y_train):
    print("message:", message)
    print("category:", category)

Streaming output truncated to the last 5000 lines.
message: ['also jdubois cannot notice the tips folder <url >']
category: sadness
message: ['hi single be running into cucumberspring124 vs springboot130 publication <url> javalangArrayStoreException sunreflectannotationTypeNotPresentExceptionProxy']
category: fear
message: ['i wish i could do this variety of work for free as well but unfortunately information technology non potential right now but it would be a dream come true to work full fourth dimension on a labor like this']
category: sadness
message: ['So 1 would just stretch that a litle bit maybe with a few notes to that effect and everything would be all good']
category: joy
message: ['Lead and tricks look awe inspiring']
category: joy
message: ['Ping ping ping Iu2019d like to get the jspm work flow for ill famed release tonight D Just 380 take a fix or is information technology my fault']
category: joy
message: ['National intelligence community an destination this with this']


**Converting Augmented Data Back into Strings for Feature Extraction**

In [ ]:
X_train = [' '.join(x) for x in X_train]

**Feature Extraction using TF-IDF**

In [ ]:
vectorizer = TfidfVectorizer()
                         
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

**Training the Dataset using Support Vector Machine (SVM)**

In [ ]:
#training the dataset using SVM
model = SVC(kernel='rbf', decision_function_shape='ovo')
model.fit(X_train, y_train)

SVC(decision_function_shape='ovo')

**Testing and Evaluating the Model**

In [ ]:
#predicting on the testing set
y_pred = model.predict(X_test)

#Generating Classification Report of the Model for Computing Precision, Recall and F1-Score for Every Six Emotion Categories
clsf_rprt = classification_report(y_test, y_pred)
print("Classification_report:\n", clsf_rprt)

Classification_report:
               precision    recall  f1-score   support

       anger       0.50      0.07      0.12        28
        fear       0.44      0.49      0.46        78
         joy       0.52      0.65      0.58        88
        love       0.33      0.22      0.27         9
     sadness       0.45      0.48      0.46        88
    surprise       0.00      0.00      0.00         9

    accuracy                           0.47       300
   macro avg       0.37      0.32      0.32       300
weighted avg       0.45      0.47      0.44       300



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#calculating precision score
precision = precision_score(y_test, y_pred, average='weighted')
print("Precision (weighted):", precision)

#calculating recall score
recall = recall_score(y_test, y_pred, average='weighted')
print("Recall (weighted):", recall)

#calculating f1_score score
f1_score = f1_score(y_test, y_pred, average='weighted')
print("f1-score (weighted):", f1_score)

#calculating accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Precision (weighted): 0.4546882102575493
Recall (weighted): 0.47
f1-score (weighted): 0.4445550507022588
Accuracy: 0.47


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Testing the Efficiency of the Model with New Textual Messages Containing Emotion**

In [ ]:
#predicting on new text data containing emotion
text_with_emotion_1 = ["Why the hell do I need to change the code again and again.."]
text_with_emotion_1 = vectorizer.transform(text_with_emotion_1)
predicted_labels = model.predict(text_with_emotion_1)
print(predicted_labels)

['anger']


In [ ]:
#predicting on new text data containing emotion
text_with_emotion_2 = ["thanks a lot guys for helping me solving the bug, i appreciate it very much"]
text_with_emotion_2 = vectorizer.transform(text_with_emotion_2)
predicted_labels = model.predict(text_with_emotion_2)
print(predicted_labels)

['love']


In [ ]:
#predicting on new text data containing emotion
text_with_emotion_3 = ["It wasn't supposed to be required, still wondering about it"]
text_with_emotion_3 = vectorizer.transform(text_with_emotion_3)
predicted_labels = model.predict(text_with_emotion_3)
print(predicted_labels)

['sadness']
